# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

In [ ]:
# Imports

In [ ]:
import warnings
# warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf

# tf.enable_eager_execution()

from matplotlib import pyplot as plt
# matplotlib.use('TkAgg')  # Use TkAgg to show images

# from PIL import Image
import glob as glob
import random
import pickle

import sys
import cv2

%matplotlib inline
# tf.executing_eagerly()

# Environment setup

In [ ]:


sys.path.append('../models/research')  # Replace with the path to TensorFlow Object Detection API
sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# Tensorflow Object Detection API

## Model preparation 

In [ ]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [ ]:
PATH_TO_LABELS = r'C:\Yuxing\traffic-sign-detection-master\gtsdb_data\gtsdb_label_map.pbtxt'

# label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
# categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
# category_index = label_map_util.create_category_index(categories)

# Directly reading the file for troubleshooting
try:
    with open(PATH_TO_LABELS, 'rb') as f:
        print("Direct read:", f.read())
except Exception as e:
    print("File read error:", str(e))

# Your existing code
print('1')
try:
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
except Exception as e:
    print("Error during label map load:", str(e))
print('2')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print('3')
category_index = label_map_util.create_category_index(categories)
print(label_map)


## Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    return np.array(image.getdata()).reshape((im_height, im_width, 3))

## Detection

In [ ]:

PATH_TO_TEST_IMAGES_DIR = 'test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

SAVE_DIR = '/test_images_detection'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)


In [ ]:
# Function to print detections
def print_detections(sess, image_np_expanded, detection_graph, image_name, confidence_threshold=0.3):
    # Get the tensors for detection
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Run the session to get the detection boxes, scores, and classes
    (boxes, scores, classes) = sess.run(
        [detection_boxes, detection_scores, detection_classes],
        feed_dict={image_tensor: image_np_expanded}
    )

    # Filter out the results with confidence scores less than the threshold
    filter_indices = np.where(scores[0] >= confidence_threshold)[0]

    # Apply the filter to boxes, scores and classes
    filtered_boxes = boxes[0][filter_indices]
    filtered_scores = scores[0][filter_indices]
    filtered_classes = classes[0][filter_indices]

    # Print the image name along with other detection results
    print(f"Image Name: {image_name}")
    print("Filtered Detection Boxes:", filtered_boxes)
    print("Filtered Detection Scores:", filtered_scores)
    print("Filtered Detection Classes:", filtered_classes)


In [ ]:

saved_detection_results = {}

# Run inference and visualization
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            saved_detection_results[image_path] = {
                'boxes': boxes,
                'scores': scores,
                'classes': classes,
                'num_detections': num_detections
            }


            # Get image name and create a new name
            image_name = os.path.basename(image_path)
            save_path = os.path.join(SAVE_DIR, f'detection_{image_name}')


            # Print and visualize detection results on adversarial image
            print_detections(sess, image_np_expanded, detection_graph, image_name)

            # Visualize and save detection results on the image
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            
            # Using matplotlib to save the image
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            plt.axis('off')
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            # plt.close()


## Algorithm 1: Adversarial pattern generation on downsampled image, using the gradient with saturator

In [ ]:
# Tuning parameters
FACTOR = 0.2
H = 5
NUM_SAMPLES = 500
NUM_ITERATIONS = 40
EPS = 400000
SAT = 50  # Saturation

SAVE_DIR = 'saved_images_1'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def downsample(image, factor=FACTOR):
    return cv2.resize(image, None, fx=factor, fy=factor, interpolation=cv2.INTER_LINEAR)

def upsample(image, original_shape):
    return cv2.resize(image, (original_shape[1], original_shape[0]), interpolation=cv2.INTER_LINEAR)


def compute_loss(sess, image_np_expanded, detection_graph):
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    
    (scores, classes) = sess.run([scores, classes], feed_dict={image_tensor: image_np_expanded})
    max_score_index = np.argmax(scores)
    score = scores[0][max_score_index]
    approx_logits = np.log(score / (1 - score))
    return approx_logits

def approximate_gradient(sess, image_np_expanded, detection_graph, h=H, num_samples=NUM_SAMPLES):
    original_loss = compute_loss(sess, image_np_expanded, detection_graph)
    grad_approx = np.zeros_like(image_np_expanded).astype(np.float32)
    
    total_pixels = np.prod(image_np_expanded.shape)
    
    # Randomly selecting pixel indices from the entire image
    indices = np.random.choice(total_pixels, num_samples, replace=False)
        
    for flat_idx in indices:
        idx = np.unravel_index(flat_idx, image_np_expanded.shape)
        perturb = np.zeros_like(image_np_expanded)
        perturb[idx] = h
        perturbed_image = image_np_expanded + perturb
        perturbed_loss = compute_loss(sess, perturbed_image, detection_graph)
        grad_approx[idx] = (perturbed_loss - original_loss) / h #  + random.uniform(-10e7, 10e7)
        
    return grad_approx

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            # Initialize
            adv_image = image_np.copy()

            # Downsample the adversarial image for approximating gradient
            adv_image_low_res = downsample(adv_image, FACTOR)
            adv_image_low_res_expanded = np.expand_dims(adv_image_low_res, axis=0)

            # save iteration images
            iteration_save_dir = os.path.join(SAVE_DIR, f'iteration_{idx}')
            if not os.path.exists(iteration_save_dir):
                os.makedirs(iteration_save_dir)

            # Iteration
            for i in range(NUM_ITERATIONS):
                print(f"Iteration {i + 1}")

                # Compute the approximate gradient on low-res image
                grad_approx_low_res = approximate_gradient(sess, adv_image_low_res_expanded, detection_graph, num_samples=NUM_SAMPLES)

                # Upsample gradient to original resolution
                grad_approx = upsample(grad_approx_low_res[0], original_shape)

                # Update adversarial image
                adv_image = adv_image + np.clip(EPS * grad_approx, -SAT, SAT)

                adv_image = np.clip(adv_image, 0, 255).astype(np.uint8)

                # Save the current adversarial image without boxes
                plt.imsave(os.path.join(iteration_save_dir, f'adv_image_iteration_{i + 1}.jpg'), adv_image)

                # Test the adversarial image on the model
                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Print detection results for the current iteration
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > 0.5:
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        print(f"Iteration {i + 1}: Detected class: {class_name}, Confidence Score: {score:.8f}")

                # Visualize the detection results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Display the current adversarial image with bounding boxes
                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()

## Algorithm 2: Adversarial pattern generation on downsampled image, using the sign of gradient

In [ ]:
f# Tuning parameters
FACTOR = 0.2
H = 5
NUM_SAMPLES = 500
NUM_ITERATIONS = 40
EPS = 25

SAVE_DIR = 'saved_image_2'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)


def approximate_gradient(sess, image_np_expanded, detection_graph, h=H, num_samples=NUM_SAMPLES):
    original_loss = compute_loss(sess, image_np_expanded, detection_graph)
    grad_approx = np.zeros_like(image_np_expanded).astype(np.float32)
    
    total_pixels = np.prod(image_np_expanded.shape)
    
    # Randomly selecting pixel indices from the entire image
    indices = np.random.choice(total_pixels, num_samples, replace=False)
        
    for flat_idx in indices:
        idx = np.unravel_index(flat_idx, image_np_expanded.shape)
        perturb = np.zeros_like(image_np_expanded)
        perturb[idx] = h
        perturbed_image = image_np_expanded + perturb
        perturbed_loss = compute_loss(sess, perturbed_image, detection_graph)
        grad_approx[idx] = (perturbed_loss - original_loss) / h 

    return grad_approx

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            # Initialize
            adv_image = image_np.copy()

            # Downsample the adversarial image for approximating gradient
            adv_image_low_res = downsample(adv_image, FACTOR)
            adv_image_low_res_expanded = np.expand_dims(adv_image_low_res, axis=0)

            # save iteration images
            iteration_save_dir = os.path.join(SAVE_DIR, f'iteration_{idx}')
            if not os.path.exists(iteration_save_dir):
                os.makedirs(iteration_save_dir)

            # Iteration
            for i in range(NUM_ITERATIONS):
                print(f"Iteration {i + 1}")

                # Compute the approximate gradient on low-res image
                grad_approx_low_res = approximate_gradient(sess, adv_image_low_res_expanded, detection_graph, num_samples=NUM_SAMPLES)

                # Upsample gradient to original resolution
                grad_approx = upsample(grad_approx_low_res[0], original_shape)
                # Update adversarial image
                adv_image = adv_image + EPS * np.sign(grad_approx)

                adv_image = np.clip(adv_image, 0, 255).astype(np.uint8)

                # Save the current adversarial image without boxes
                plt.imsave(os.path.join(iteration_save_dir, f'adv_image_iteration_{i + 1}.jpg'), adv_image)

                # Test the adversarial image on the model
                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Print detection results for the current iteration
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > 0.5:
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        print(f"Iteration {i + 1}: Detected class: {class_name}, Confidence Score: {score:.8f}")

                # Visualize the detection results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Display the current adversarial image with bounding boxes
                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()

## Algorithm 3: Adversarial pattern generation on downsampled image, using the gradient without saturator

In [ ]:
# Tuning parameters
FACTOR = 0.2
H = 5
NUM_SAMPLES = 500
NUM_ITERATIONS = 40
EPS = 200000

SAVE_DIR = 'saved_image_3'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)


def approximate_gradient(sess, image_np_expanded, detection_graph, h=H, num_samples=NUM_SAMPLES):
    original_loss = compute_loss(sess, image_np_expanded, detection_graph)
    grad_approx = np.zeros_like(image_np_expanded).astype(np.float32)
    
    total_pixels = np.prod(image_np_expanded.shape)
    
    # Randomly selecting pixel indices from the entire image
    indices = np.random.choice(total_pixels, num_samples, replace=False)
        
    for flat_idx in indices:
        idx = np.unravel_index(flat_idx, image_np_expanded.shape)
        perturb = np.zeros_like(image_np_expanded)
        perturb[idx] = h
        perturbed_image = image_np_expanded + perturb
        perturbed_loss = compute_loss(sess, perturbed_image, detection_graph)
        grad_approx[idx] = (perturbed_loss - original_loss) / h 

    return grad_approx

# Main execution
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            original_shape = image_np.shape

            # Initialize
            adv_image = image_np.copy()

            # Downsample the adversarial image for approximating gradient
            adv_image_low_res = downsample(adv_image, FACTOR)
            adv_image_low_res_expanded = np.expand_dims(adv_image_low_res, axis=0)

            # save iteration images
            iteration_save_dir = os.path.join(SAVE_DIR, f'iteration_{idx}')
            if not os.path.exists(iteration_save_dir):
                os.makedirs(iteration_save_dir)

            # Iteration
            for i in range(NUM_ITERATIONS):
                print(f"Iteration {i + 1}")

                # Compute the approximate gradient on low-res image
                grad_approx_low_res = approximate_gradient(sess, adv_image_low_res_expanded, detection_graph, num_samples=NUM_SAMPLES)

                # Upsample gradient to original resolution
                grad_approx = upsample(grad_approx_low_res[0], original_shape)

                # Update adversarial image
                adv_image = adv_image + EPS * grad_approx


                adv_image = np.clip(adv_image, 0, 255).astype(np.uint8)

                # Save the current adversarial image without boxes
                plt.imsave(os.path.join(iteration_save_dir, f'adv_image_iteration_{i + 1}.jpg'), adv_image)

                # Test the adversarial image on the model
                adv_image_expanded = np.expand_dims(adv_image, axis=0)
                (iter_boxes, iter_scores, iter_classes, iter_num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: adv_image_expanded}
                )
                
                # Print detection results for the current iteration
                for j, score in enumerate(np.squeeze(iter_scores)):
                    if score > 0.5:
                        class_id = int(np.squeeze(iter_classes)[j])
                        class_name = category_index[class_id]['name']
                        print(f"Iteration {i + 1}: Detected class: {class_name}, Confidence Score: {score:.8f}")

                # Visualize the detection results
                adv_image_visual = adv_image.copy()
                vis_util.visualize_boxes_and_labels_on_image_array(
                    adv_image_visual, 
                    np.squeeze(iter_boxes),
                    np.squeeze(iter_classes).astype(np.int32),
                    np.squeeze(iter_scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6,
                    min_score_thresh=0.5
                )

                # Display the current adversarial image with bounding boxes
                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(adv_image_visual) 
                plt.title(f"Iteration {i + 1} Detection Results")
                plt.show()